### Experiments using IMLY ###

This notebook contains experimental runs of IMLY with different datasets.  
The readings of these experiments can be referred to in this [sheet](https://docs.google.com/spreadsheets/d/1E5jcq2w42gN8bMIaeaRJpAdhgSVN-2XDJ_YTHe4qfwY/edit?usp=sharing).

---

### Dataset  #1

#### Diabetes dataset

In [1]:
import experiment_automation_script
from os import path
import pandas as pd

dataset_info = experiment_automation_script.get_dataset_info("diabetes")
url = "../data/diabetes.csv" if path.exists("../data/diabetes.csv") else dataset_info['url']
data = pd.read_csv(url, delimiter=",", header=None, index_col=False)

X = data.iloc[:,:-1]
Y = data.iloc[:,-1]

experiment_automation_script.dopify(dataset_info, 'linear_regression', X, Y, 0.60)

Using TensorFlow backend.
100%|██████████| 3/3 [00:03<00:00,  1.35s/it]


Scan Finished!
Epoch 1/1
176/176 [==============================] - ETA: 0s - loss: 157.1494 - mean_absolute_error: 157.149 - 0s 710us/step - loss: 151.9254 - mean_absolute_error: 151.9254


AttributeError: 'SklearnKerasRegressor' object has no attribute 'evaluate'

---

### Dataset  #2

#### UCI Abalone dataset

In [2]:
import experiment_automation_script
import pandas as pd
import numpy as np
from os import path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


dataset_info = experiment_automation_script.get_dataset_info("uci_abalone")

names = ["sex", "length", "diameter", "height", "whole weight",
        "shucked weight", "viscera weight", "shell weight", "rings"]
url = "../data/abalone.data.csv" if path.exists("../data/abalone.data.csv") else dataset_info['url']
data = pd.read_csv(url, delimiter=",", header=None, names=names, index_col=False)
data.head()

# Check for columns that contain missing values #
col_names = data.columns

num_data = data.shape[0]

categorical_col = ['sex']
for col in categorical_col:
    b, c = np.unique(data[col], return_inverse=True)
    data[col] = c

    
# Filter dataset to contain 'rings' 9 and 10 #
data = data[data['rings'].isin([9,10])]
data['rings'] = data['rings'].map({9: 0, 10: 1})


feature_list = names[:7]
X = data.loc[:, feature_list]
Y = data[['rings']]


experiment_automation_script.dopify(dataset_info, 'logistic_regression', X, Y, 0.60)

C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
100%|██████████| 1/1 [00:01<00:00,  1.45s/it]


Scan Finished!
Epoch 1/1
794/794 [==============================] - ETA:  - 0s 59us/step


---

### Dataset  #3

#### UCI Iris dataset

In [1]:
import experiment_automation_script
import pandas as pd
import numpy as np
from os import path

dataset_name = "uci_iris"
dataset_info = experiment_automation_script.get_dataset_info(dataset_name)

url = "../data/iris.csv" if path.exists("../data/iris.csv") else dataset_info['url']
data = pd.read_csv(url , delimiter=",", header=None, index_col=False)
class_name,index = np.unique(data.iloc[:,-1],return_inverse=True)
data.iloc[:,-1] = index
data = data.loc[data[4] != 2]
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]

experiment_automation_script.dopify(dataset_info, 'logistic_regression', X, Y, 0.60)

Using TensorFlow backend.


Keras classifier chosen


100%|██████████| 1/1 [00:12<00:00, 12.74s/it]


Scan Finished!


TypeError: __call__() missing 1 required positional argument: 'inputs'

In [14]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)
np.random.seed(7)

def create_model():
    model = Sequential()
    model.add(Dense(1,input_dim=4,activation='sigmoid'))

    # Compile the model #

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    return model
    

model = KerasClassifier(build_fn=create_model, epochs=200, batch_size=10)
model.fit(x_train, y_train)

scores = model.score(x_test, y_test)
scores

Epoch 1/200
40/40 [==============================] - ETA: 1s - loss: 0.9877 - acc: 0.700 - 0s 12ms/step - loss: 1.3947 - acc: 0.5500
Epoch 2/200
40/40 [==============================] - ETA: 0s - loss: 1.7549 - acc: 0.400 - 0s 391us/step - loss: 1.3838 - acc: 0.5500
Epoch 3/200
40/40 [==============================] - ETA: 0s - loss: 1.2139 - acc: 0.600 - 0s 391us/step - loss: 1.3706 - acc: 0.5500
Epoch 4/200
40/40 [==============================] - ETA: 0s - loss: 1.2532 - acc: 0.600 - 0s 390us/step - loss: 1.3582 - acc: 0.5500
Epoch 5/200
40/40 [==============================] - ETA: 0s - loss: 1.8960 - acc: 0.300 - 0s 781us/step - loss: 1.3481 - acc: 0.5500
Epoch 6/200
40/40 [==============================] - ETA: 0s - loss: 1.6283 - acc: 0.400 - 0s 782us/step - loss: 1.3347 - acc: 0.5500
Epoch 7/200
40/40 [==============================] - ETA: 0s - loss: 1.5119 - acc: 0.500 - 0s 390us/step - loss: 1.3230 - acc: 0.5500
Epoch 8/200
40/40 [==============================] - ETA: 0s - 

Epoch 62/200
40/40 [==============================] - ETA: 0s - loss: 0.9020 - acc: 0.600 - 0s 391us/step - loss: 0.9359 - acc: 0.5500
Epoch 63/200
40/40 [==============================] - ETA: 0s - loss: 1.0838 - acc: 0.400 - 0s 390us/step - loss: 0.9303 - acc: 0.5500
Epoch 64/200
40/40 [==============================] - ETA: 0s - loss: 1.2043 - acc: 0.300 - 0s 391us/step - loss: 0.9260 - acc: 0.5500
Epoch 65/200
40/40 [==============================] - ETA: 0s - loss: 0.8818 - acc: 0.600 - 0s 391us/step - loss: 0.9203 - acc: 0.5500
Epoch 66/200
40/40 [==============================] - ETA: 0s - loss: 1.1933 - acc: 0.300 - 0s 391us/step - loss: 0.9156 - acc: 0.5500
Epoch 67/200
40/40 [==============================] - ETA: 0s - loss: 0.8611 - acc: 0.600 - 0s 390us/step - loss: 0.9101 - acc: 0.5500
Epoch 68/200
40/40 [==============================] - ETA: 0s - loss: 1.0028 - acc: 0.500 - 0s 390us/step - loss: 0.9055 - acc: 0.5500
Epoch 69/200
40/40 [==============================] - E

40/40 [==============================] - ETA: 0s - loss: 0.6378 - acc: 0.600 - 0s 391us/step - loss: 0.6820 - acc: 0.5500
Epoch 123/200
40/40 [==============================] - ETA: 0s - loss: 0.7650 - acc: 0.400 - 0s 391us/step - loss: 0.6787 - acc: 0.5500
Epoch 124/200
40/40 [==============================] - ETA: 0s - loss: 0.7781 - acc: 0.400 - 0s 391us/step - loss: 0.6752 - acc: 0.5500
Epoch 125/200
40/40 [==============================] - ETA: 0s - loss: 0.7509 - acc: 0.400 - 0s 391us/step - loss: 0.6718 - acc: 0.5500
Epoch 126/200
40/40 [==============================] - ETA: 0s - loss: 0.7017 - acc: 0.500 - 0s 391us/step - loss: 0.6683 - acc: 0.5500
Epoch 127/200
40/40 [==============================] - ETA: 0s - loss: 0.7666 - acc: 0.400 - 0s 391us/step - loss: 0.6650 - acc: 0.5500
Epoch 128/200
40/40 [==============================] - ETA: 0s - loss: 0.6777 - acc: 0.500 - 0s 391us/step - loss: 0.6617 - acc: 0.5500
Epoch 129/200
40/40 [==============================] - ETA: 0s

40/40 [==============================] - ETA: 0s - loss: 0.5132 - acc: 0.600 - 0s 391us/step - loss: 0.5084 - acc: 0.7000
Epoch 183/200
40/40 [==============================] - ETA: 0s - loss: 0.5198 - acc: 0.700 - 0s 391us/step - loss: 0.5059 - acc: 0.7500
Epoch 184/200
40/40 [==============================] - ETA: 0s - loss: 0.4556 - acc: 0.800 - 0s 391us/step - loss: 0.5036 - acc: 0.7750
Epoch 185/200
40/40 [==============================] - ETA: 0s - loss: 0.4901 - acc: 0.800 - 0s 390us/step - loss: 0.5012 - acc: 0.8000
Epoch 186/200
40/40 [==============================] - ETA: 0s - loss: 0.5417 - acc: 0.800 - 0s 391us/step - loss: 0.4990 - acc: 0.8750
Epoch 187/200
40/40 [==============================] - ETA: 0s - loss: 0.5641 - acc: 0.900 - 0s 0us/step - loss: 0.4966 - acc: 0.8750
Epoch 188/200
40/40 [==============================] - ETA: 0s - loss: 0.4609 - acc: 0.800 - 0s 390us/step - loss: 0.4942 - acc: 0.9000
Epoch 189/200
40/40 [==============================] - ETA: 0s -

0.9333333174387614

In [5]:
model.get_params()

{'build_fn': <function __main__.create_model()>}

---

### Dataset  #4

#### UCI Adult salary dataset

In [6]:
import experiment_automation_script
import pandas as pd
import numpy as np
from os import path

dataset_name = "uci_adult_salary"
dataset_info = experiment_automation_script.get_dataset_info(dataset_name)


names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
         'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
         'hours-per-week', 'native-country', 'target']
url = "../data/iris.csv" if path.exists("../data/dataset.csv.csv") else dataset_info['url']
data = pd.read_csv(url, delimiter=" ", header=None, names=names)


data = data[data["workclass"] != "?"]
data = data[data["occupation"] != "?"]
data = data[data["native-country"] != "?"]

# Convert categorical fields #
categorical_col = ['workclass', 'education', 'marital-status', 'occupation',
                   'relationship', 'race', 'sex', 'native-country', 'target']

for col in categorical_col:
    b, c = np.unique(data[col], return_inverse=True)
    data[col] = c

feature_list = names[:14]
# Test train split #
X = data.loc[:, feature_list]
Y = data[['target']]

experiment_automation_script.dopify(dataset_info, 'logistic_regression', X, Y, 0.60)

# Split the dataset into test and train datasets

# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
100%|██████████| 1/1 [00:33<00:00, 33.26s/it]


Scan Finished!
Epoch 1/1
27134/27134 [==============================] - ETA: 27 - ETA: 1 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 38us/step


---

### Dataset  #5

#### UCI Ad dataset

In [8]:
import experiment_automation_script
import pandas as pd
import numpy as np
from os import path
from sklearn.preprocessing import LabelEncoder

dataset_name = "uci_ad"
dataset_info = experiment_automation_script.get_dataset_info(dataset_name)

url = "../data/ad.data.csv" if path.exists("../data/dataset.csv.csv") else dataset_info['url']
data = pd.read_csv(url, delimiter=",", header=None, index_col=False)

# Check for columns that contain missing values #

data = data.applymap(lambda val: np.nan if str(val).strip() == '?' else val)
data = data.dropna()


# Label encoding #

lb = LabelEncoder()
Y = lb.fit_transform(data.iloc[:, -1])

X = data.iloc[:,:-1]

# Normalize the X values #
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
X = pd.DataFrame(X)
Y = pd.DataFrame(Y)

experiment_automation_script.dopify(dataset_info, 'logistic_regression', X, Y, 0.60)

C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
100%|██████████| 1/1 [00:02<00:00,  2.51s/it]


Scan Finished!
Epoch 1/1
1416/1416 [==============================] - ETA:  - ETA:  - 0s 61us/step


---

### Dataset  #6

#### UCI Mushroom dataset

In [3]:
from sklearn.model_selection import train_test_split
import copy

model_name = 'linear_regression'
model_mappings = {
    'linear_regression': 'LinearRegression',
    'logistic_regression': 'LogisticRegression'
}

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

for key, value in model_mappings.items():
    if key == model_name:
        name = value

module = __import__('sklearn.linear_model', fromlist=[name])
imported_module = getattr(module, name)
model = imported_module

primal_model = model()

# Primal
primal_model.fit(x_train, y_train)
primal_model.__class__.__name__

'LinearRegression'

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

diabetes = datasets.load_diabetes()

# Use only one feature
diabetes_X = diabetes.data
# sc = StandardScaler()
# diabetes.data = sc.fit_transform(diabetes.data)

X = diabetes.data
Y = diabetes.target

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

# # Split the data into training/testing sets
# x_train = diabetes_X[:-20]
# x_test = diabetes_X[-20:]

# # Split the targets into training/testing sets
# y_train = diabetes.target[:-20]
# y_test = diabetes.target[-20:]


In [23]:
# temp_data = np.column_stack([X,Y])
# np.savetxt("diabetes.csv", temp_data, delimiter=",")

In [3]:
import winmltools
winmltools.__file__

'C:\\Users\\shakk\\Anaconda2\\envs\\py36\\lib\\site-packages\\winmltools\\__init__.py'

In [9]:
import onnxmltools

def f1(**kwargs):
    params_json = json.load(open('../imly/architectures/sklearn/params.json'))
    params = params_json['params']
    kwargs.setdefault('params', params)
    kwargs.setdefault('x_train', np.array([[1], [2]]))

    model = Sequential()
    model.add(Dense(kwargs['params']['first_neuron'],
                    input_dim=kwargs['x_train'].shape[1],
                    activation=kwargs['params']['activation']))

    model.compile(optimizer=kwargs['params']['optimizer'],
                  loss=kwargs['params']['losses'],
                  metrics=['acc'])
    onnx_model = onnxmltools.convert_keras(model, target_opset=8)
    print(type(model))
    onnx_model
    return onnx_model

In [10]:
import json
import numpy as np
from keras import Sequential
from keras.layers import Dense

model = f1()

The maximum opset needed by this model is only 7.


<class 'keras.engine.sequential.Sequential'>


In [19]:
type(model)
# cross check import (f1p1 and f2p2 combination) - Is it possible to edit after the export-import flow

onnx.onnx_ml_pb2.ModelProto

In [18]:
import onnx
onnx.save(model, './onnx_model.onnx')

---

## Test bed ##

In [40]:
mapping = { "KerasClassifier": {
    "LogisticRegression": True
},
 "KerasRegressor": {
     "LinearRegression": True
 }
}

name = "LinearRegression"

for key, value in mapping.items():
    test = mapping[key]
    print(test)

{'LogisticRegression': True}
{'LinearRegression': True}
